## Running prompts on OpenAI

In [ ]:
%pip install markdown
%pip install annoy
%pip install openai

In [ ]:
from openai import OpenAI # OpenAI API
import json
import requests # to download some resources
import os # file operations
import numpy as np # linear algebra
import pandas as pd # data processing
from markdown import markdown # to render markdown
from IPython.display import Markdown
import annoy # Approximate Nearest Neighbors Oh Yeah for fast searching
import pickle
from annoy import AnnoyIndex

In [ ]:
# read in our api key
with open('../api-keys/our_api_key.txt', 'r') as file:
    api_key = file.read().replace('\n', '')

# read in finns api key (the one we'll use for testing)
with open('../api-keys/finns_api_key.txt', 'r') as file:
    finns_api_key = file.read().replace('\n', '')

In [ ]:
# Setting the OpenAI
client = OpenAI(api_key=finns_api_key) # using finns for testing

#### Sample of countries

In [ ]:
# Sample of countries
sample_countries = ['Finland','Netherlands','Denmark','Norway', 'Chile', 'Bangladesh']

# Folder with raw PDFs
folder_mds = "../data/3-naps-md"

# Importing MDs files
for file in os.listdir(folder_mds):
    for country in sample_countries:
        if country in file:  
            with open(os.path.join(folder_mds, file), "r", encoding="utf-8") as md_file:
                content = md_file.read()
            globals()[country.lower()] = content # Saving the MD file in lowercase


### 1. Using the WHOLE document

In [15]:
country_test = netherlands
country_name = "netherlands"

In [16]:
system_prompt = """
You are an assistant that extracts and summarizes national action plans for antimicrobial resistance (NAPs). Your task is to answer the following questions concisely while relying strictly on the document content. 

For each question, provide:
1. A 'yes' or 'no' answer.
2. The specific text from the document that supports the answer.

Here are the questions:

1. What is the period considered for the NAP? (Provide the exact years or date range)
2. Does the NAP include any strategy/policy/goal/action related to the priority sector of "Human Health"? (yes/no + supporting text)
3. Does the NAP include any strategy/policy/goal/action related to the priority sector of "Animal Health"? (yes/no + supporting text)
4. Does the NAP include any strategy/policy/goal/action related to the priority sector of "Environment"? (yes/no + supporting text)
5. Does the NAP include any strategy/policy/goal/action related to the priority sector of "Agriculture/Food Security"? (yes/no + supporting text)
6. Does the NAP include any mechanisms for progress reporting to track how its objectives are being met, such as an annual or semi-annual progress report, a dashboard displaying the status, or similar documents/tools? (yes/no + supporting text)
7. Does the NAP include any specific, time-bound targets to track progress? (yes/no + supporting text)
8. Does the NAP specify any budget allocation assigned for the strategies/policies/goals/actions directed at tackling AMR? (yes/no + supporting text)
9. Does the NAP establish a multisectoral committee or task force for AMR coordination? (yes/no + supporting text)
10. Does the NAP include any strategy/policy/goal/action related to training and educational initiatives to increase awareness about AMR – in school curricula or professional training programs for doctors and pharmacists? (yes/no + supporting text)
11. Does the NAP include any strategy/policy/goal/action related to public awareness campaigns in combating AMR, such as media involvement and community engagement? (yes/no + supporting text)
12. Does the NAP include any strategy/policy/goal/action related to international collaboration and support for AMR initiatives, such as partnerships with organizations like WHO, FAO, and OIE? (yes/no + supporting text)
13. Does the NAP include any strategy/policy/goal/action related to data collection and analysis for AMR surveillance, such as developing surveillance systems, data collection tools, setting up laboratories for AMR monitoring, or other similar actions? (yes/no + supporting text)
14. Does the NAP include any strategy/policy/goal/action related to developing the capacity to detect and report newly emerged resistance that may constitute a public health emergency of international concern, as required by the International Health Regulations (2005)? (yes/no + supporting text)
15. Does the NAP include any strategy/policy/goal/action related to specific prevention strategies such as sanitation, hygiene measures, and infection control? (yes/no + supporting text)
16. Does the NAP include any strategy/policy/goal/action related to implementing the standards outlined in the OIE Terrestrial and Aquatic Animal Health Codes or the FAO/WHO Codex Alimentarius Code of Practice to Minimize and Contain Antimicrobial Resistance? (yes/no + supporting text)
17. Does the NAP include any strategy/policy/goal/action related to regulating the distribution, prescription, and dispensation of antibiotics, such as developing or maintaining essential medicine lists for antibiotics? (yes/no + supporting text)
18. Does the NAP include any strategy/policy/goal/action related to strengthening the legislative and regulatory framework for AMR, including laws and regulations for antimicrobial use? (yes/no + supporting text)
19. Does the NAP include any strategy/policy/goal/action related to requiring the rational use of antibiotics through regulations/laws? (yes/no + supporting text)
20. Does the NAP include any strategy/policy/goal/action related to participating in international collaborative research to support the development of new medicines, diagnostic tools, and vaccines? (yes/no + supporting text)

Format the response as a JSON object:
{
  "period": "XXXX-YYYY",
  "questions": [
    {
      "question": "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?",
      "answer": "yes",
      "supporting_text": "The NAP mentions specific actions in the human health sector, including..."
    },
    ...
  ]
}
"""

In [17]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0,  # Ensures deterministic output
    top_p=0.5,  # Controls diversity
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": country_test}
    ]
)

# Print the structured JSON response
print(response.choices[0].message.content)

{
  "period": "2015-2019",
  "questions": [
    {
      "question": "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?",
      "answer": "yes",
      "supporting_text": "The approach to antibiotic resistance addresses all domains where human health is threatened by antibiotic-resistant bacteria; healthcare, animals, food and environment."
    },
    {
      "question": "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Animal Health'?",
      "answer": "yes",
      "supporting_text": "The goal is reduction and prudent use of antibiotics in animal farming in order to limit the development of resistance where possible."
    },
    {
      "question": "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Environment'?",
      "answer": "yes",
      "supporting_text": "Knowledge about the development of, the occurrence of and the spread of antibiotic resistance in p

In [18]:
import json

# Extract response content
response_json = json.loads(response.choices[0].message.content)

# Save response as JSON
output_path = f"../data/4-summary-responses-json/summary_response_{country_name}.json"
with open(output_path, "w") as f:
    json.dump(response_json, f, indent=4)

print(f"Saved response to {output_path}")

Saved response to ../data/4-summary-responses-json/summary_response_netherlands.json


---

### 2. Calculating performance

#### Performance: Bangladesh

In [ ]:
country_name = "bangladesh"

In [ ]:
# Import results
with open(f"../data/4-summary-responses-json/summary_response_{country_name}.json", "r") as f:
    model_answers = json.load(f)

# Load the annotations
annotations_df = pd.read_excel('../data/5-annotations/annotations.xlsx')
annotations_df['question'] = annotations_df['question'].astype(str)
annotations_df['answer'] = annotations_df['answer'].astype(str)

# Filter for the current country
annotations_df = annotations_df[annotations_df['country'] == country_name]

# Extract period and questions from the model response
model_answers = [{"country": country_name, "question": "period_start", "answer": response_json["period"].split("-")[0]}]
model_answers.append({"country": country_name, "question": "period_end", "answer": response_json["period"].split("-")[1]})

for q in response_json["questions"]:
    model_answers.append({
        "country": country_name,
        "question": q["question"],
        "answer": q["answer"]
       
    })

# Convert to DataFrame
model_answers_df = pd.DataFrame(model_answers)
model_answers_df['question'] = model_answers_df['question'].astype(str)

question_mapping = {
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?": "1",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Animal Health'?": "2",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Environment'?": "3",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Agriculture/Food Security'?": "4",
    "Does the NAP include any mechanisms for progress reporting to track how its objectives are being met, such as an annual or semi-annual progress report, a dashboard displaying the status, or similar documents/tools?": "5",
    "Does the NAP include any specific, time-bound targets to track progress?": "6",
    "Does the NAP specify any budget allocation assigned for the strategies/policies/goals/actions directed at tackling AMR?": "7",
    "Does the NAP establish a multisectoral committee or task force for AMR coordination?": "8",
    "Does the NAP include any strategy/policy/goal/action related to training and educational initiatives to increase awareness about AMR – in school curricula or professional training programs for doctors and pharmacists?": "9",
    "Does the NAP include any strategy/policy/goal/action related to public awareness campaigns in combating AMR, such as media involvement and community engagement?": "10",
    "Does the NAP include any strategy/policy/goal/action related to international collaboration and support for AMR initiatives, such as partnerships with organizations like WHO, FAO, and OIE?": "11",
    "Does the NAP include any strategy/policy/goal/action related to data collection and analysis for AMR surveillance, such as developing surveillance systems, data collection tools, setting up laboratories for AMR monitoring, or other similar actions?": "12",
    "Does the NAP include any strategy/policy/goal/action related to developing the capacity to detect and report newly emerged resistance that may constitute a public health emergency of international concern, as required by the International Health Regulations (2005)?": "13",
    "Does the NAP include any strategy/policy/goal/action related to specific prevention strategies such as sanitation, hygiene measures, and infection control?": "14",
    "Does the NAP include any strategy/policy/goal/action related to implementing the standards outlined in the OIE Terrestrial and Aquatic Animal Health Codes or the FAO/WHO Codex Alimentarius Code of Practice to Minimize and Contain Antimicrobial Resistance?": "15",
    "Does the NAP include any strategy/policy/goal/action related to regulating the distribution, prescription, and dispensation of antibiotics, such as developing or maintaining essential medicine lists for antibiotics?": "16",
    "Does the NAP include any strategy/policy/goal/action related to strengthening the legislative and regulatory framework for AMR, including laws and regulations for antimicrobial use?": "17",
    "Does the NAP include any strategy/policy/goal/action related to requiring the rational use of antibiotics through regulations/laws?": "18",
    "Does the NAP include any strategy/policy/goal/action related to participating in international collaborative research to support the development of new medicines, diagnostic tools, and vaccines?": "19"
}

model_answers_df["question"] = model_answers_df["question"].replace(question_mapping)

# Merge based on country and question
merged_df = pd.merge(annotations_df, model_answers_df, on=["country", "question"], suffixes=('_gs', '_llm'))

In [ ]:
# Saving file .xlsx with results
folder_per = f"../data/6-performance/performance_{country_name}.xlsx"

# Verifying if the file already exists
if not os.path.exists(folder_per):
    # Si el archivo no existe, guardar el DataFrame
    merged_df.to_excel(folder_per, index=False, engine="openpyxl")
else:
    # If file do exist, read it and merge
    performance_df = pd.read_excel(folder_per)
    performance_select_df=performance_df.loc[:, ['country', 'question', 'answer_llm_chunking','supporting_chunks']]
    merged2_df = merged_df.join(performance_select_df, how='inner',lsuffix='_left', rsuffix='_right')
    # Dropping repeated columns
    merged2_df = merged2_df.drop(columns=['country_right', 'question_right'])
    # Changing names of the columns
    merged2_df = merged2_df.rename(columns={'country_left': 'country', 'question_left': 'question'})
    merged2_df.to_excel(folder_per, index=False, engine="openpyxl")
merged2_df.head()


In [ ]:
merged_df

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert categorical yes/no answers to binary (1 for yes, 0 for no)
merged_df['gs_binary'] = merged_df['answer_gs'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))
merged_df['llm_binary'] = merged_df['answer_llm'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))

# Drop rows with NaN values in either gs_binary or llm_binary for accurate calculations
yesno_df = merged_df[~merged_df['question'].isin(["period_start", "period_end"])]
yesno_df = yesno_df.dropna(subset=['gs_binary', 'llm_binary'])

# Performance for yes/no questions
accuracy = accuracy_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
precision = precision_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
recall = recall_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
f1 = f1_score(yesno_df['gs_binary'], yesno_df['llm_binary'])

# Performance for period extraction (period_start, period_end)
period_matches = merged_df[merged_df['question'].isin(["period_start", "period_end"])]

# For period extraction, we don't need binary conversion, so we directly compare the answers
period_matches['period_match'] = period_matches['answer_gs'] == period_matches['answer_llm']

# Calculate period extraction accuracy as the percentage of exact matches
period_accuracy = period_matches['period_match'].mean()  # Exact match percentage

# Questions where the model failed
questions_failed = merged_df[merged_df['gs_binary'] != merged_df['llm_binary']]['question'].unique()

# Print results
print(f"Accuracy (Yes/No Questions): {accuracy:.2f}")
print(f"Precision (Yes/No Questions): {precision:.2f}")
print(f"Recall (Yes/No Questions): {recall:.2f}")
print(f"F1 Score (Yes/No Questions): {f1:.2f}")
print(f"Period Extraction Accuracy: {period_accuracy:.2f}")
print(f"Questions where the model failed: {questions_failed}")

#### Performance: Netherlands

In [ ]:
country_name = "netherlands"

In [35]:
# Import results
with open(f"../data/4-summary-responses-json/summary_response_{country_name}.json", "r") as f:
    model_answers = json.load(f)

# Load the annotations
annotations_df = pd.read_excel('../data/5-annotations/annotations.xlsx')
annotations_df['question'] = annotations_df['question'].astype(str)
annotations_df['answer'] = annotations_df['answer'].astype(str)

# Filter for the current country
annotations_df = annotations_df[annotations_df['country'] == country_name]

# Extract period and questions from the model response
model_answers = [{"country": country_name, "question": "period_start", "answer": response_json["period"].split("-")[0]}]
model_answers.append({"country": country_name, "question": "period_end", "answer": response_json["period"].split("-")[1]})

for q in response_json["questions"]:
    model_answers.append({
        "country": country_name,
        "question": q["question"],
        "answer": q["answer"],
        "supporting_text":q["supporting_text"]
    })

# Convert to DataFrame
model_answers_df = pd.DataFrame(model_answers)
model_answers_df['question'] = model_answers_df['question'].astype(str)

question_mapping = {
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?": "1",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Animal Health'?": "2",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Environment'?": "3",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Agriculture/Food Security'?": "4",
    "Does the NAP include any mechanisms for progress reporting to track how its objectives are being met, such as an annual or semi-annual progress report, a dashboard displaying the status, or similar documents/tools?": "5",
    "Does the NAP include any specific, time-bound targets to track progress?": "6",
    "Does the NAP specify any budget allocation assigned for the strategies/policies/goals/actions directed at tackling AMR?": "7",
    "Does the NAP establish a multisectoral committee or task force for AMR coordination?": "8",
    "Does the NAP include any strategy/policy/goal/action related to training and educational initiatives to increase awareness about AMR – in school curricula or professional training programs for doctors and pharmacists?": "9",
    "Does the NAP include any strategy/policy/goal/action related to public awareness campaigns in combating AMR, such as media involvement and community engagement?": "10",
    "Does the NAP include any strategy/policy/goal/action related to international collaboration and support for AMR initiatives, such as partnerships with organizations like WHO, FAO, and OIE?": "11",
    "Does the NAP include any strategy/policy/goal/action related to data collection and analysis for AMR surveillance, such as developing surveillance systems, data collection tools, setting up laboratories for AMR monitoring, or other similar actions?": "12",
    "Does the NAP include any strategy/policy/goal/action related to developing the capacity to detect and report newly emerged resistance that may constitute a public health emergency of international concern, as required by the International Health Regulations (2005)?": "13",
    "Does the NAP include any strategy/policy/goal/action related to specific prevention strategies such as sanitation, hygiene measures, and infection control?": "14",
    "Does the NAP include any strategy/policy/goal/action related to implementing the standards outlined in the OIE Terrestrial and Aquatic Animal Health Codes or the FAO/WHO Codex Alimentarius Code of Practice to Minimize and Contain Antimicrobial Resistance?": "15",
    "Does the NAP include any strategy/policy/goal/action related to regulating the distribution, prescription, and dispensation of antibiotics, such as developing or maintaining essential medicine lists for antibiotics?": "16",
    "Does the NAP include any strategy/policy/goal/action related to strengthening the legislative and regulatory framework for AMR, including laws and regulations for antimicrobial use?": "17",
    "Does the NAP include any strategy/policy/goal/action related to requiring the rational use of antibiotics through regulations/laws?": "18",
    "Does the NAP include any strategy/policy/goal/action related to participating in international collaborative research to support the development of new medicines, diagnostic tools, and vaccines?": "19"
}

model_answers_df["question"] = model_answers_df["question"].replace(question_mapping)

# Merge based on country and question
merged_df = pd.merge(annotations_df, model_answers_df, on=["country", "question"], suffixes=('_gs', '_llm_full_document'))

In [38]:
# Saving file .xlsx with results
folder_per = f"../data/6-performance/performance_{country_name}.xlsx"

# Verifying if the file already exists
if not os.path.exists(folder_per):
    # Si el archivo no existe, guardar el DataFrame
    merged_df.to_excel(folder_per, index=False, engine="openpyxl")
else:
    # If file do exist, read it and merge
    performance_df = pd.read_excel(folder_per)
    performance_select_df=performance_df.loc[:, ['country', 'question', 'answer_llm_chunking','supporting_chunks']]
    merged2_df = merged_df.join(performance_select_df, how='inner',lsuffix='_left', rsuffix='_right')
    # Dropping repeated columns
    merged2_df = merged2_df.drop(columns=['country_right', 'question_right'])
    # Changing names of the columns
    merged2_df = merged2_df.rename(columns={'country_left': 'country', 'question_left': 'question'})
    merged2_df.to_excel(folder_per, index=False, engine="openpyxl")
merged2_df.head()


,country,question,answer_gs,comment_input_for_protocol,supporting_text_when_true_answer_is_yes_but_prediction_is_no,answer_llm_full_document,supporting_text,answer_llm_chunking,supporting_chunks
0,netherlands,period_start,2015,NaN,NaN,2015,NaN,2016,NaN
1,netherlands,period_end,2019,NaN,NaN,2019,NaN,2016,NaN
2,netherlands,1,yes,NaN,Actors agree to the following goals: The numbe...,yes,The approach to antibiotic resistance addresse...,yes,['The approach to antibiotic resistance we pro...
3,netherlands,2,yes,NaN,The reduction of antibiotic use has flattened....,yes,The goal is reduction and prudent use of antib...,yes,['The main focus within the approach to antibi...
4,netherlands,3,yes,NaN,"Despite these uncertainties, the research by Z...",yes,"Knowledge about the development of, the occurr...",yes,['The approach to antibiotic resistance we pro...


In [39]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert categorical yes/no answers to binary (1 for yes, 0 for no)
merged_df['gs_binary'] = merged_df['answer_gs'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))
merged_df['llm_binary'] = merged_df['answer_llm_full_document'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))

# Drop rows with NaN values in either gs_binary or llm_binary for accurate calculations
yesno_df = merged_df[~merged_df['question'].isin(["period_start", "period_end"])]
yesno_df = yesno_df.dropna(subset=['gs_binary', 'llm_binary'])

# Performance for yes/no questions
accuracy = accuracy_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
precision = precision_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
recall = recall_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
f1 = f1_score(yesno_df['gs_binary'], yesno_df['llm_binary'])

# Performance for period extraction (period_start, period_end)
period_matches = merged_df[merged_df['question'].isin(["period_start", "period_end"])]

# For period extraction, we don't need binary conversion, so we directly compare the answers
period_matches['period_match'] = period_matches['answer_gs'] == period_matches['answer_llm_full_document']

# Calculate period extraction accuracy as the percentage of exact matches
period_accuracy = period_matches['period_match'].mean()  # Exact match percentage

# Questions where the model failed
questions_failed = merged_df[merged_df['gs_binary'] != merged_df['llm_binary']]['question'].unique()

# Print results
print(f"Accuracy (Yes/No Questions): {accuracy:.2f}")
print(f"Precision (Yes/No Questions): {precision:.2f}")
print(f"Recall (Yes/No Questions): {recall:.2f}")
print(f"F1 Score (Yes/No Questions): {f1:.2f}")
print(f"Period Extraction Accuracy: {period_accuracy:.2f}")
print(f"Questions where the model failed: {questions_failed}")

Accuracy (Yes/No Questions): 0.68
Precision (Yes/No Questions): 0.65
Recall (Yes/No Questions): 1.00
F1 Score (Yes/No Questions): 0.79
Period Extraction Accuracy: 1.00
Questions where the model failed: ['period_start' 'period_end' '4' '12' '13' '15' '17' '18']


/var/folders/j7/mgqwccss6cl5clys8j0_ckvm0000gn/T/ipykernel_22582/3550396538.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period_matches['period_match'] = period_matches['answer_gs'] == period_matches['answer_llm_full_document']


#### Performance: Chile

In [ ]:
country_name = "chile"

In [ ]:
# Import results
with open(f"../data/4-summary-responses-json/summary_response_{country_name}.json", "r") as f:
    model_answers = json.load(f)

# Load the annotations
annotations_df = pd.read_excel('../data/5-annotations/annotations.xlsx')
annotations_df['question'] = annotations_df['question'].astype(str)
annotations_df['answer'] = annotations_df['answer'].astype(str)

# Filter for the current country
annotations_df = annotations_df[annotations_df['country'] == country_name]

# Extract period and questions from the model response
model_answers = [{"country": country_name, "question": "period_start", "answer": response_json["period"].split("-")[0]}]
model_answers.append({"country": country_name, "question": "period_end", "answer": response_json["period"].split("-")[1]})

for q in response_json["questions"]:
    model_answers.append({
        "country": country_name,
        "question": q["question"],
        "answer": q["answer"]
    })

# Convert to DataFrame
model_answers_df = pd.DataFrame(model_answers)

question_mapping = {
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?": "1",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Animal Health'?": "2",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Environment'?": "3",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Agriculture/Food Security'?": "4",
    "Does the NAP include any mechanisms for progress reporting to track how its objectives are being met, such as an annual or semi-annual progress report, a dashboard displaying the status, or similar documents/tools?": "5",
    "Does the NAP include any specific, time-bound targets to track progress?": "6",
    "Does the NAP specify any budget allocation assigned for the strategies/policies/goals/actions directed at tackling AMR?": "7",
    "Does the NAP establish a multisectoral committee or task force for AMR coordination?": "8",
    "Does the NAP include any strategy/policy/goal/action related to training and educational initiatives to increase awareness about AMR – in school curricula or professional training programs for doctors and pharmacists?": "9",
    "Does the NAP include any strategy/policy/goal/action related to public awareness campaigns in combating AMR, such as media involvement and community engagement?": "10",
    "Does the NAP include any strategy/policy/goal/action related to international collaboration and support for AMR initiatives, such as partnerships with organizations like WHO, FAO, and OIE?": "11",
    "Does the NAP include any strategy/policy/goal/action related to data collection and analysis for AMR surveillance, such as developing surveillance systems, data collection tools, setting up laboratories for AMR monitoring, or other similar actions?": "12",
    "Does the NAP include any strategy/policy/goal/action related to developing the capacity to detect and report newly emerged resistance that may constitute a public health emergency of international concern, as required by the International Health Regulations (2005)?": "13",
    "Does the NAP include any strategy/policy/goal/action related to specific prevention strategies such as sanitation, hygiene measures, and infection control?": "14",
    "Does the NAP include any strategy/policy/goal/action related to implementing the standards outlined in the OIE Terrestrial and Aquatic Animal Health Codes or the FAO/WHO Codex Alimentarius Code of Practice to Minimize and Contain Antimicrobial Resistance?": "15",
    "Does the NAP include any strategy/policy/goal/action related to regulating the distribution, prescription, and dispensation of antibiotics, such as developing or maintaining essential medicine lists for antibiotics?": "16",
    "Does the NAP include any strategy/policy/goal/action related to strengthening the legislative and regulatory framework for AMR, including laws and regulations for antimicrobial use?": "17",
    "Does the NAP include any strategy/policy/goal/action related to requiring the rational use of antibiotics through regulations/laws?": "18",
    "Does the NAP include any strategy/policy/goal/action related to participating in international collaborative research to support the development of new medicines, diagnostic tools, and vaccines?": "19"
}

model_answers_df["question"] = model_answers_df["question"].replace(question_mapping)

# Merge based on country and question
merged_df = pd.merge(annotations_df, model_answers_df, on=["country", "question"], suffixes=('_gs', '_llm'))

In [ ]:
# Saving file .xlsx with results
folder_per = f"../data/6-performance/performance_{country_name}.xlsx"

# Verifying if the file already exists
if not os.path.exists(folder_per):
    # Si el archivo no existe, guardar el DataFrame
    merged_df.to_excel(folder_per, index=False, engine="openpyxl")
else:
    # If file do exist, read it and merge
    performance_df = pd.read_excel(folder_per)
    performance_select_df=performance_df.loc[:, ['country', 'question', 'answer_llm_chunking','supporting_chunks']]
    merged2_df = merged_df.join(performance_select_df, how='inner',lsuffix='_left', rsuffix='_right')
    # Dropping repeated columns
    merged2_df = merged2_df.drop(columns=['country_right', 'question_right'])
    # Changing names of the columns
    merged2_df = merged2_df.rename(columns={'country_left': 'country', 'question_left': 'question'})
    merged2_df.to_excel(folder_per, index=False, engine="openpyxl")
merged2_df.head()


In [ ]:
merged_df

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert categorical yes/no answers to binary (1 for yes, 0 for no)
merged_df['gs_binary'] = merged_df['answer_gs'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))
merged_df['llm_binary'] = merged_df['answer_llm'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))

# Drop rows with NaN values in either gs_binary or llm_binary for accurate calculations
yesno_df = merged_df[~merged_df['question'].isin(["period_start", "period_end"])]
yesno_df = yesno_df.dropna(subset=['gs_binary', 'llm_binary'])

# Performance for yes/no questions
accuracy = accuracy_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
precision = precision_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
recall = recall_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
f1 = f1_score(yesno_df['gs_binary'], yesno_df['llm_binary'])

# Performance for period extraction (period_start, period_end)
period_matches = merged_df[merged_df['question'].isin(["period_start", "period_end"])]

# For period extraction, we don't need binary conversion, so we directly compare the answers
period_matches['period_match'] = period_matches['answer_gs'] == period_matches['answer_llm']

# Calculate period extraction accuracy as the percentage of exact matches
period_accuracy = period_matches['period_match'].mean()  # Exact match percentage

# Questions where the model failed
questions_failed = merged_df[merged_df['gs_binary'] != merged_df['llm_binary']]['question'].unique()

# Print results
print(f"Accuracy (Yes/No Questions): {accuracy:.2f}")
print(f"Precision (Yes/No Questions): {precision:.2f}")
print(f"Recall (Yes/No Questions): {recall:.2f}")
print(f"F1 Score (Yes/No Questions): {f1:.2f}")
print(f"Period Extraction Accuracy: {period_accuracy:.2f}")
print(f"Questions where the model failed: {questions_failed}")

In [ ]:
# see how many tokens that last query used
response.usage